In [14]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pickle
import os 
import numpy as np
import pandas as pd
import getpass
import os



input_file = "/home/jeans/nvaitc/battery_timeseries/resources/raw/test_bat_data.pkl"

with open(input_file, "rb") as file:
    test_dict = pickle.load(file)
    
for bat in test_dict:
    # print(bat)
    trimmed_arr = np.trim_zeros(test_dict[bat]['q_d_n'], 'b')
    test_dict[bat]['trimmed_q_d_n'] = list(trimmed_arr)
    test_dict[bat]['gradient_q_d_n'] = list(np.gradient(trimmed_arr))
    trimmed_arr = trimmed_arr[len(trimmed_arr)//2:]
    # print(np.gradient(trimmed_arr).shape)
    # print(np.mean(np.gradient(trimmed_arr)))
    
    test_dict[bat]['mean_backhalf_grad'] = np.mean(np.gradient(trimmed_arr))

In [16]:
test_key = 'b3c45'
sample_datapoint = test_dict[test_key]

mean_bh_grad = sample_datapoint['mean_backhalf_grad']
cycle = sample_datapoint['cycle']
trimmed_q_d_n = sample_datapoint['trimmed_q_d_n']
print("sample batteryID: ", test_key)
print("sample cycle: ", cycle)
print("sample mean backhalf gradient: ", mean_bh_grad)
print("trimmed_q_d_n: ", trimmed_q_d_n)
# print("sample trimmed q_d_n: ", trimmed_q_d_n)

sample batteryID:  b3c45
sample cycle:  1802
sample mean backhalf gradient:  -0.00017983621021486679
trimmed_q_d_n:  [1.068801760673523, 1.069548487663269, 1.070114254951477, 1.070421814918518, 1.0706182718276978, 1.0709116458892822, 1.0711549520492554, 1.0710127353668213, 1.0711963176727295, 1.0713307857513428, 1.0712343454360962, 1.0713157653808594, 1.0713841915130615, 1.0714848041534424, 1.0717002153396606, 1.0716644525527954, 1.0716475248336792, 1.0717617273330688, 1.0716674327850342, 1.0716513395309448, 1.0715487003326416, 1.0715796947479248, 1.0717144012451172, 1.0717065334320068, 1.0716166496276855, 1.071985125541687, 1.0706861019134521, 1.0708918571472168, 1.0720105171203613, 1.072011947631836, 1.0719444751739502, 1.0721019506454468, 1.0720264911651611, 1.0719643831253052, 1.0719362497329712, 1.0717873573303223, 1.0706042051315308, 1.070672631263733, 1.0719764232635498, 1.0719618797302246, 1.0718334913253784, 1.071921706199646, 1.0717788934707642, 1.0717793703079224, 1.07197868

In [17]:
os.environ["NEO4J_URI"] = "neo4j+s://3b31837b.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(refresh_schema=False)

graph.refresh_schema()
print(graph.schema)

Node properties:
Battery {batteryID: STRING, mean_backhalf_grad: FLOAT, degradation_cycle: INTEGER}
Cycle {batteryID: STRING, cycle_number: INTEGER, discharge_capacity: FLOAT, gradient: FLOAT}
ChargingProtocol {protocolName: STRING}
Relationship properties:
NEXT_CYCLE {gradient: FLOAT}
The relationships:
(:Battery)-[:HAS_CYCLE]->(:Cycle)
(:Cycle)-[:NEXT_CYCLE]->(:Cycle)
(:ChargingProtocol)-[:USED_BY]->(:Battery)


In [18]:
# mean_bh_grad_range = 0.1
# cycle_range = 50
# #query db for similar node (:Battery) with similar mean_backhalf_grad and cycle
# # aslo return new attributes showing how close the match is 
# query = f"""
# MATCH (b:Battery)
# WHERE abs(b.mean_backhalf_grad - {mean_bh_grad}) < {mean_bh_grad_range}
# AND abs(b.degradation_cycle - {cycle}) < {cycle_range}
# RETURN b
# """
# results = graph.query(query)
# for result in results:
#     print(result['b'])
#     print("mean_bh_grad_diff", abs(result['b']['mean_backhalf_grad'] - mean_bh_grad))
#     print("cycle_diff", abs(result['b']['degradation_cycle'] - cycle))

# #query db for similar node (:Battery) with similar mean_backhalf_grad and cycle

# Define query ranges
mean_bh_grad_range = 1
cycle_range = 5000

# Query DB for similar nodes and return ChargingProtocol too
query = f"""
MATCH (b:Battery)-[:USED_BY]-(cp:ChargingProtocol)
WHERE abs(b.mean_backhalf_grad - {mean_bh_grad}) < {mean_bh_grad_range}
AND abs(b.degradation_cycle - {cycle}) < {cycle_range}
RETURN b, cp,
abs(b.mean_backhalf_grad - {mean_bh_grad}) AS mean_bh_grad_diff,
abs(b.degradation_cycle - {cycle}) AS cycle_diff
ORDER BY mean_bh_grad_diff + cycle_diff ASC
"""
print(query)
# Run the query
results = graph.query(query)

# Process and print the top-1 result
for result in results:
    battery = result['b']
    protocol = result['cp']
    mean_bh_grad_diff = result['mean_bh_grad_diff']
    cycle_diff = result['cycle_diff']
    
    print("Top match:")
    print("Battery:", battery)
    print("ChargingProtocol:", protocol)
    print("mean_bh_grad_diff:", mean_bh_grad_diff)
    print("cycle_diff:", cycle_diff)



MATCH (b:Battery)-[:USED_BY]-(cp:ChargingProtocol)
WHERE abs(b.mean_backhalf_grad - -0.00017983621021486679) < 1
AND abs(b.degradation_cycle - 1802) < 5000
RETURN b, cp,
abs(b.mean_backhalf_grad - -0.00017983621021486679) AS mean_bh_grad_diff,
abs(b.degradation_cycle - 1802) AS cycle_diff
ORDER BY mean_bh_grad_diff + cycle_diff ASC

Top match:
Battery: {'batteryID': 'b1c1', 'mean_backhalf_grad': -0.0001512054598629199, 'degradation_cycle': 2161}
ChargingProtocol: {'protocolName': '3.6C(80%)-3.6C'}
mean_bh_grad_diff: 2.8630750351946893e-05
cycle_diff: 359
Top match:
Battery: {'batteryID': 'b1c3', 'mean_backhalf_grad': -0.0002491650143041584, 'degradation_cycle': 1435}
ChargingProtocol: {'protocolName': '4C(80%)-4C'}
mean_bh_grad_diff: 6.932880408929164e-05
cycle_diff: 367
Top match:
Battery: {'batteryID': 'b1c5', 'mean_backhalf_grad': -0.0003298082609194395, 'degradation_cycle': 1074}
ChargingProtocol: {'protocolName': '4.4C(80%)-4.4C'}
mean_bh_grad_diff: 0.0001499720507045727
cycle_di

In [19]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
if os.environ.get("NVIDIA_API_KEY") is None:
    os.environ["NVIDIA_API_KEY"] = input()

# Set up NVIDIA ChatNVIDIA LLM
llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    api_key=os.environ["NVIDIA_API_KEY"],
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
)

# Query the graph database
query = """
MATCH (b:Battery)-[:HAS_CYCLE]->(c:Cycle)
RETURN b.batteryID AS batteryID, c.cycle_number AS cycle, c.discharge_capacity AS capacity
LIMIT 10
"""
# result = graph.query(query)

# Format results for the LLM
data = [{"batteryID": record["batteryID"], "cycle": record["cycle"], "capacity": record["capacity"]} for record in result]

# Pass to LLM for further analysis
prompt = f'''Format this text for presentation slide and correct my grammar:\n\nbattery_dataset : dict() with batteryID as key
Example 
->battery_dataset['b1c1'] : dict with 6 keys ['q_d_n', 'trimmed_q_d_n', 'cycle', 'mean_backhalf_grad', 'gradient_q_d_n']

Attributes
- cycle : int 
- q_d_n : array of float showing the discharge capacity left? of each cycle. (unused)
- trimmed_q_d_n : q_d_n but trimmed 0s off.
- gradient_q_d_n : array of float showing the grad between discharge capacity left (length == cycle).
- mean_backhalf_grad : floating point number representing the mean of gradient from all of the backhalf element of the array in q_d_n`
-charging_policy : str'''
print(prompt)
for chunk in llm.stream([{"role": "user", "content": prompt}]):
    print(chunk.content, end="")

TypeError: string indices must be integers